# 07 - Upload Models to S3 Storage

This notebook uploads the trained models and their scalers to an S3-compatible object storage service, making them available for deployment and serving in production environments.


## Import Required Libraries

Import boto3 for S3 operations and os for environment variable access.


In [1]:
import os
import boto3

## Load S3 Configuration from Environment

Retrieve AWS S3 credentials and endpoint information from environment variables.


In [2]:
key_id = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
endpoint = os.getenv("AWS_S3_ENDPOINT")
bucket_name = os.getenv("AWS_S3_BUCKET")

## Create S3 Client

Initialize the boto3 S3 client with the configured credentials and endpoint.


In [3]:
s3_client = boto3.client(
    "s3",
    aws_access_key_id=key_id,
    aws_secret_access_key=secret_key,
    endpoint_url=endpoint,
    use_ssl=True
)

## Upload Stress Detection Model

Define a function to recursively upload directory contents to S3, then upload the stress detection model files.


In [4]:
local_model_path = 'models/serving/battery_stress_model'
s3_prefix = 'models/serving/battery_stress_model'

def upload_directory_to_s3(local_path, bucket, s3_prefix):
    uploaded_files = []
    
    for root, dirs, files in os.walk(local_path):
        for file in files:
            local_file = os.path.join(root, file)
            relative_path = os.path.relpath(local_file, local_path)
            s3_key = os.path.join(s3_prefix, relative_path).replace('\\', '/')
            
            s3_client.upload_file(local_file, bucket, s3_key)
            uploaded_files.append(s3_key)
    
    return uploaded_files

uploaded = upload_directory_to_s3(local_model_path, bucket_name, s3_prefix)

## Upload Stress Model Scaler

Upload the StandardScaler used for the stress detection model to ensure consistent data preprocessing during inference.


In [ ]:
scaler_local_path = 'models/stress_scaler.pkl'
scaler_s3_key = 'models/serving/battery_stress_model/scaler.pkl'

if os.path.exists(scaler_local_path):
    s3_client.upload_file(scaler_local_path, bucket_name, scaler_s3_key)

## Upload Time-to-Failure Model and Scaler

Upload the TTF model files and its corresponding scaler to S3 storage.


In [6]:
local_ttf_model_path = 'models/serving/battery_ttf_model'
s3_ttf_prefix = 'models/serving/battery_ttf_model'

uploaded_ttf = upload_directory_to_s3(local_ttf_model_path, bucket_name, s3_ttf_prefix)

ttf_scaler_local_path = 'models/ttf_scaler.pkl'
ttf_scaler_s3_key = 'models/serving/battery_ttf_model/ttf_scaler.pkl'

if os.path.exists(ttf_scaler_local_path):
    s3_client.upload_file(ttf_scaler_local_path, bucket_name, ttf_scaler_s3_key)